In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import re
import json

In [3]:
# only letters and numbers
letters_numbers_pat = re.compile("[^a-zA-Z0-9ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]+")

# only letters
letters_pat = re.compile("[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ]+")

# only numbers
numbers_pat = re.compile("[^0-9]+")

In [4]:
url = "https://booksy.com/pl-pl/s/13750_wroclaw?businessesPage={page_num}"
base_url = "https://booksy.com"

In [5]:
def write_json(data):
    with open("./data/booksy_wroclaw.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)


def open_json(file: str):
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

In [ ]:
data = []
idx = 0

for page_num in range(1, 160):

    try:
        page_url = url.format(page_num=page_num)
        response = requests.get(page_url)
        bs = BeautifulSoup(response.text, "html.parser")

        businesses = bs.find_all(
            class_="purify_jFNrjJsLiLW+6sHFuYDiWA== purify_gvbw+GpawACpHIzUwLBo8A== purify_CA-Z3uLarGK6crMd3kMgRw=="
        )

        for business in businesses:
            business_dict = {}
            business_dict["id"] = idx

            name = bs.find(class_="purify_3UeKw5MV+NerIrQNs8HSXA== purify_IbX1Bb-sl2ffhqyi9xUH1A==")
            clean_name = name.text.replace("\n", "").replace("  ", " ").strip()
            business_dict["name"] = clean_name

            href = business.find("a")["href"]
            business_url = base_url + href
            response = requests.get(business_url)
            bs = BeautifulSoup(response.text, "html.parser")

            address_parent_div = (
                bs.find(
                    class_="purify_prm7MfDXczhTZvcY5KwOuA== purify_Sardy6hfiet162IZ2pYFPA== purify_m9mNOPjpHD0tNTW6GC+hEw=="
                )
                .text.replace("\n", "")
                .strip()
            )
            address = address_parent_div.split("div")[0].replace("\n", "").strip()
            business_dict["address"] = address

            rating = business.find(
                class_="purify_aVm421MES2COps1la9BuOQ== purify_3dZ+zycQyUtqfDxWbTGTlw== purify_kq4BZr36QXoLgkAnN95TWw=="
            ).text.strip()
            business_dict["rating"] = rating
            
            services = bs.find_all(class_="purify_TDNkQVqTWE1e9+GIIAxpIA==")
            prices = bs.find_all(
                class_="purify_s8fylQf1Z9K4N9kwPLHJaA== purify_IbX1Bb-sl2ffhqyi9xUH1A== purify_f9tJlQGTtqZe8Hg0f7oQhg=="
            )
            business_dict["services"] = []
            for service, price in zip(services, prices):
                try:
                    service_dict = {}
                    service_clean = (
                        letters_pat.sub(" ", service.text)
                        .replace("\n", "")
                        .replace("  ", " ")
                        .strip()
                        .lower()
                    )
                    price_clean = (
                        price.text.replace("\n", "").replace("  ", " ").replace("\xa0", " ").strip()
                    )

                    service_dict["price"] = price_clean
                    service_dict["name"] = service_clean
                    business_dict["services"].append(service_dict)
                except Exception as e:
                    print(f"Error: {e}")
                    continue

            idx += 1

            # try:
            #     driver.get(page_url)
            #     html = driver.page_source
            #     soup = BeautifulSoup(html, "html.parser")
            #     soup_str = str(soup)
            #     description = soup_str[
            #         soup_str.find("description:") : soup_str.find(",business_categories")
            #     ]
            #     business_dict["description"] = description
            # except Exception as e:
            #     print(f"Error: {e}")
            #     business_dict["description"] = None

            data.append(business_dict)

    except Exception as e:
        print(f"Error: {e}")
        continue

write_json(data)